# __텍스트로 텍스트 검색하기__

- 튜토리얼 난이도: ★★☆☆☆
- 읽는데 걸리는 시간: 7분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_search/search_text_by_text.ipynb
- 참고 문서: [Naver sentiment movie corpus v1.0](https://github.com/e9t/nsmc#naver-sentiment-movie-corpus-v10), [단어 임베딩: 어휘의 의미(LEXICAL SEMANTICS)를 인코딩하기](https://tutorials.pytorch.kr/beginner/nlp/word_embeddings_tutorial.html), [컴퓨터가 바라보는 문자](https://heung-bae-lee.github.io/2020/01/16/NLP_01/)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">텍스트 수치화 기술 이해하기</h4>
    <p>컴퓨터는 사람이 쓰는 언어(자연어)를 곧바로 입력으로 받을 수 없습니다. 때문에 자연어를 기계가 인식할 수 있는 수치 데이터로 변환하는 과정이 필요합니다. 자연어 처리분야에서 임베딩이란 이렇게 사람이 쓰는 자연어를 기계가 이해할 수 있는 숫자형태인 vector로 바꾼 결과 혹은 그 일련의 과정 전체를 의미합니다.</p>
</div>

자연어의 임베딩을 구하는 기법은 크게 통계적인 기법과, 인공신경망 기반 기법으로 구분되고, 세부적으로도 더 나눌 수 있습니다. ThanoSQL에서는 인공신경망 기반의 [자가학습모델(Self-Supervised Learning Model)](https://ko.wikipedia.org/wiki/%EC%9E%90%EA%B8%B0_%EC%A7%80%EB%8F%84_%ED%95%99%EC%8A%B5)을 사용하여, 텍스트를 잘 표현하는 벡터를 스스로 학습하는 방법을 제공합니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 이번 튜토리얼에서는 영화 리뷰 데이터를 사용합니다. 데이터는 영화 리뷰 텍스트와, 라벨값으로 구성되어있지만, 자가학습법을 사용해 훈련하는 예제를 보여드리기 위해 라벨값은 사용하지 않습니다. 10,000개의 영화 리뷰 데이터를 학습하여, 입력 문장과 유사한 문장 찾기, 문장의 키워드 추출하기를 해보겠습니다.</p>
</div>

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/paas/workspace/lab/)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET nsmc_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY nsmc_train
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/nsmc_data/nsmc_sample_train.csv'

Success


In [4]:
%%thanosql
COPY nsmc_test
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/nsmc_data/nsmc_sample_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"if_exists": 동일 이름의 테이블이 존재하는 경우 처리하는 방법 설정. 오류 발생, 기존 테이블에 추가, 기존 테이블 대체 (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

텍스트 수치화 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __nsmc_train__ 테이블을 사용합니다. __nsmc_train__ 테이블은 NAVER Sentiment Movie Corpus 영화 리뷰 데이터 및 라벨 정보의 일부가 담겨 있는 테이블입니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM nsmc_train
LIMIT 5

,id,document,label
0,9465891,저퀄리티. 뭐하나 장점이 없음.,0
1,9792014,10점 만점에 1점 !!! 최악이네 하아...,0
2,3795348,누가 억지눈물 잘흘리나 자기네들끼리 대결하는 작품,0
3,1129393,신도 싫고 인간도 싫다,1
4,4678610,추억의 명작만화,1


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>nsmc_train</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>id: 데이터의 식별 번호</li>
        <li>document: 영화 리뷰 데이터</li>
        <li>label: 라벨값</li>
    </ul>
</div>

## __2. 텍스트 수치화 모델 생성__

이전 단계에서 확인한 __nsmc_train__ 테이블을 사용하여 텍스트 수치화 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>nsmc_text_search_model</strong>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간: 2 min)

In [6]:
%%thanosql
BUILD MODEL nsmc_text_search_model
USING SBERTKo
OPTIONS (
    text_col='document',
    overwrite=True
    )
AS
SELECT *
FROM nsmc_train

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>nsmc_text_search_model</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>SBERTKo</strong>를 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (str, default: 'text')</li>
            <li>"max_epochs": 텍스트 수치화 모델을 생성하기 위한 데이터 세트 학습 횟수 (int, optional, default: 1)</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

다음 "__CONVERT USING__ " 쿼리 구문을 실행하여 __nsmc_test__ 텍스트들을 수치화 합니다. 수치화 된 결과는 __nsmc_test__ 테이블에 사용자가 옵션으로 지정한 이름(default: 'convert_result')의 컬럼에 저장됩니다.

In [7]:
%%thanosql
CONVERT USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    batch_size=32,
    result_col='convert_result'
    )
AS
SELECT *
FROM nsmc_test

,id,document,label,convert_result
0,9181084,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[b'\xc9', b'\x06', b'\x8a', b'\xbe', b'O', b'\..."
1,6318649,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[b')', b'\xac', b'k', b'\xbf', b'\xb3', b'\xf8..."
2,5718332,유쾌하고 신나는 스피드 코미디.,1,"[b'm', b'.', b'\x11', b'\xbf', b'M', b'\xa6', ..."
3,8063675,정말 따뜻했던 드라마ㅠㅠ,1,"[b'\xc6', b'\xd1', b'\x97', b'>', b'\x10', b'\..."
4,10229366,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[b'\xe1', b'S', b'\xbd', b'\xbe', b'M', b'\x1c..."
...,...,...,...,...
1995,5382227,30년간 본 수천편의 드라마중 최고는 이 작품,1,"[b'\xfb', b'\xfe', b'\xd3', b'\xbe', b'\x94', ..."
1996,4584773,빛돌을 이길 5명의 용사들의 패배,1,"[b'\xbe', b'\xd3', b'\x1f', b'\xbf', b'k', b'\..."
1997,2559484,빌머레이의 굳은 표정에 모든것이 살아있다.. 짐자무시 최고!,1,"[b'c', b'B', b'V', b'\xbe', b'K', b'\xbf', b'\..."
1998,9867081,이런영화에 투자하지 않는것이 진정한 한국영화의 발전을위한 초석,0,"[b'\xc1', b'f', b'\x17', b'>', b'*', b',', b'-..."


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" 쿼리 구문은 <strong>nsmc_text_search_model</strong> 모델을 텍스트 수치화를 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (str, default: 'text')</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"result_col": 데이터 테이블에서 수치화된 결과를 담을 컬럼 이름 (str, optional, default: 'convert_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 텍스트 수치화 모델을 사용하여 유사한 문서 검색__

이번 단계에서는 __nsmc_text_search_model__ 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서를 검색합니다.

In [8]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT
    USING nsmc_text_search_model
    OPTIONS (
        search_by='text',
        search_input='영화 내용이 불만족스러웠다',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS 
    SELECT *
    FROM nsmc_test
    )

,document,label,score
0,영화 전개가 너무 지루해요.,0,0.644379
1,너무 작위적인 스토리 전개..실망스러운 영화ㅠ,0,0.630725
2,마지막이 허무하긴했지만 재밌었음,1,0.621673
3,내가 생각했던 스토리와 조금 달랐지만 신선하고 독특했던 영화였다.,1,0.614647
4,시나리오는 한없이 후진데다가 썩은 동선이 가득한 영화,0,0.612108
5,잔잔한 영화 보기 불편할수도 있음,1,0.610900
6,영화가 왜 망한줄 알겠다...취지는 좋았으나 남자주연의 연기력보다 답답한전개가 보는...,0,0.603517
7,중간중간 짤린기분의 스토리도 많고 완전 허무,0,0.603137
8,너무나 평범한 상황극의 짜집기 보는내내 예상은적중했다,0,0.599501
9,이연걸을 썼으면 그만큼을 보여줬어야 했다 짜임새에서도 아쉬움이 많은 영화,0,0.596645


In [9]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT
    USING nsmc_text_search_model
    OPTIONS (
        search_by='text',
        search_input='기분이 좋아지는 작품',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS
    SELECT *
    FROM nsmc_test
    )

,document,label,score
0,보고나면 기분 좋은 영화,1,0.749621
1,몰입도 잘되고 좋은 영화,1,0.692085
2,감동적인 영화,1,0.688612
3,뻔한 스토리지만 과거 추억을 생각나게 해주는 영화 재미있었어요,1,0.657871
4,재미있었어요,1,0.654349
5,아름다운 영화 여운이 깊은영화,1,0.649069
6,심영이후 감동적인 작품,1,0.638167
7,재밌다,1,0.632471
8,재밌다,1,0.632471
9,너무 재밌다,1,0.631208


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|text|audio|video]</strong>" 쿼리 구문은 검색하고자 하는 이미지|텍스트|오디오|비디오 파일을 정의합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 검색 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"search_by": 검색할 때 사용할 이미지|텍스트|오디오|비디오 타입 (str)</li>
            <li>"search_input": 검색할 때 사용할 입력값 (str)</li>
            <li>"emb_col": 데이터 테이블에서 수치화된 결과를 담은 컬럼 (str)</li>
            <li>"result_col": 데이터 테이블에서 검색 결과를 담을 컬럼 이름 (str, optional, default: 'search_result')</li>
            <li>"top_k": 반환할 행의 수. None을 입력할 시 데이터 테이블 전체를 반환 (int, optional, default: 1000)</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __4. 텍스트 수치화 모델을 사용하여 문서에서 키워드 추출__

이번 단계에서는 __nsmc_text_search_model__ 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서에서 키워드를 추출합니다.

In [10]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT *
FROM nsmc_test
LIMIT 10 OFFSET 40

,id,document,label,convert_result,keyword
0,7928591,"일단 엠씨 역량 부족이 가장 큰 문제인듯. 시간 없다고, 재미없다고 게스트 말 끊는...",0,"[b'\x0b', b'\x91', b'\xb6', b'\xbe', b'^', b'a...","{'keyword': ['안듣고 걍한마디하라고하고 끝ㅋㅋㅋ엠씨가노답임', '이야기하..."
1,8380896,방금 슬쩍 봤는데 '인삼' 처럼 생긴 '산삼' 암수가 서로 토닥이다가 주인공에게 잡...,0,"[b'b', b'\xe4', b'x', b'\xbf', b'\xe5', b'r', ...","{'keyword': ['주인공에게 잡힘 ㅋㅋ', '처럼 생긴 산삼', '산삼 암수..."
2,8914833,신세계에 비교하는 거 진심 빡친다. 신성모독,1,"[b'Y', b'\xb7', b'\x97', b'\xbd', b'd', b'\xa5...","{'keyword': ['진심 빡친다 신성모독', '신세계에 비교하는 진심', '비..."
3,9702804,킬링타임 최고다 열자채워,1,"[b'Z', b'\x13', b'\xa9', b'>', b'\xba', b'D', ...","{'keyword': ['킬링타임 최고다 열자채워', '최고다 열자채워', '킬링타..."
4,10097178,명불허전 선동영화 10자,0,"[b'&', b'\xb5', b'\xa9', b'\xbe', b'\x92', b'!...","{'keyword': ['명불허전 선동영화 10자', '명불허전 선동영화', '선동..."
5,9245590,결말이 마음에 든다. 소설을 먼저 읽었던 터라 결말이 마음에 든다. 세상의 모든 정...,1,"[b'z', b'\xc7', b'3', b'\xbe', b'\xf3', b'N', ...","{'keyword': ['읽었던 터라 결말이', '결말이 마음에 든다', '모든 정..."
6,4227156,기대했는데.. 시나리오는 ...어떻게 결말이 그렇지?..요즘 막장 드라마 영향인가..,0,"[b'\x9c', b'@', b'\xe8', b'\xbd', b'\x18', b'l...","{'keyword': ['막장 드라마 영향인가', '어떻게 결말이 그렇지', '기대..."
7,7313784,인기연예인 몇명 출현한다고 재미있는 영화냐 잡것들아긴급조치19호가 그렇게 만들고 망...,0,"[b'p', b'\x1f', b'<', b'\xbd', b'E', b'\xf6', ...","{'keyword': ['재미있는 영화냐 잡것들아긴급조치19호가', '만들고 망한거..."
8,3582981,좋았음!!,1,"[b'e', b'\xe8', b'\xe8', b'>', b'\x1b', b'u', ...","{'keyword': ['좋았음'], 'score': [0.7279]}"
9,9906269,멜로와 스릴러를 이렇게 조합할수 있다니 ...,1,"[b')', b'\xbc', b'\xd0', b'\xbe', b'0', b'=', ...","{'keyword': ['스릴러를 이렇게 조합할수', '멜로와 스릴러를 이렇게', ..."


In [11]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        use_stopwords=True
        )
    AS
    SELECT *
    FROM nsmc_test
    LIMIT 10
    )

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[한번 봐야하는, 영화인 같네요, 봐야하는 영화인, 같네요 ㅎㅎ, 한번]","[0.6943, 0.6489, 0.622, 0.6193, 0.4805]"
1,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[럽라인은작가빙의된니끼한조연과 여주는남자갖고논질나쁜여자, 권선징악은안드로메다로 럽라...","[0.8648, 0.8151, 0.6957, 0.6888, 0.5718]"
2,유쾌하고 신나는 스피드 코미디.,1,"[유쾌하고 신나는, 신나는 스피드, 스피드 코미디, 유쾌하고, 신나는]","[0.788, 0.7437, 0.7207, 0.6636, 0.651]"
3,정말 따뜻했던 드라마ㅠㅠ,1,"[따뜻했던 드라마ㅠㅠ, 정말 따뜻했던, 따뜻했던, 드라마ㅠㅠ, 정말]","[0.9187, 0.8187, 0.7074, 0.6793, 0.6004]"
4,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[영화자체만을 놓고보면, 연출력이 그냥, 살인마를 주인공으로, 배우들의 연기는, 내...","[0.6047, 0.4843, 0.3922, 0.386, 0.3858]"
5,느와르에 멜로에 신파극까지······. 짬뽕이 아니라 꿀꿀이죽,0,"[신파극까지 짬뽕이, 느와르에, 짬뽕이 아니라, 아니라 꿀꿀이죽, 멜로에]","[0.554, 0.3941, 0.3893, 0.3594, 0.3061]"
6,다시 보고 싶네요~,1,"[보고 싶네요, 다시 보고, 다시, 싶네요, 보고]","[0.7833, 0.7104, 0.5793, 0.5476, 0.5176]"
7,짱이야... 진짜 극장을 나오는데 멋진 소설을 한권 읽은 듯한 기분을 느낌. 재미는...,1,"[ㅠㅠㅠ 꼭보시길, 짱이야, 감동까지 철철, 진짜 극장을, 읽은 듯한]","[0.6106, 0.5383, 0.4968, 0.3915, 0.2726]"
8,유일하게 아들과 같이볼수있는 만화영화,1,"[같이볼수있는 만화영화, 아들과 같이볼수있는, 같이볼수있는, 유일하게 아들과, 만화영화]","[0.8279, 0.7409, 0.6606, 0.6262, 0.5835]"
9,남자라면 봐라 꼭봐라 두번봐라 세번봐라 계속봐라,1,"[꼭봐라 두번봐라, 두번봐라 세번봐라, 세번봐라 계속봐라, 남자라면 봐라, 봐라]","[0.8564, 0.8499, 0.8297, 0.6199, 0.4722]"


In [12]:
%%thanosql
SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, json_array_elements(keyword -> 'score') AS score
FROM (
    SEARCH KEYWORD 
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        use_stopwords=True,
        threshold=0.5
        )
    AS
    SELECT *
    FROM nsmc_test
    LIMIT 10
    )

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,한번 봐야하는,0.6943
1,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,영화인 같네요,0.6489
2,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,봐야하는 영화인,0.6220
3,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,같네요 ㅎㅎ,0.6193
4,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,럽라인은작가빙의된니끼한조연과 여주는남자갖고논질나쁜여자,0.8648
5,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,권선징악은안드로메다로 럽라인은작가빙의된니끼한조연과,0.8151
6,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,여주는남자갖고논질나쁜여자,0.6957
7,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,럽라인은작가빙의된니끼한조연과,0.6888
8,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,권선징악은안드로메다로,0.5718
9,유쾌하고 신나는 스피드 코미디.,1,유쾌하고 신나는,0.7880


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
            <ul>
                <li>"lang": 사용할 언어 (str, optional, 'ko'|'en', default: 'ko')</li>
                <li>"text_col": 텍스트가 있는 열 이름 (str, default: 'text')</li>
                <li>"ngram_range": 키워드의 최소 단어 수와 최대 단어수 예) [1, 3]. 대부분의 상황에서 키워드는 최대 단어 수에 맞춰 추출됨 (list[int, int], optional, default: [1, 2])</li>
                <li>"top_n": 추출할 키워드의 수, 유사도가 높은 순서대로 (int, optional, default: 5)</li>
                <li>"diversity": 추출될 키워드의 다양성. 높을 수록 기존에 추출된 키워드와 유사한 키워드는 다시 추출되지 않음 0 <= diversity <= 1 (float, optional, default: 0.5)</li>
                <li>"use_stopwords": 큰 의미가 없는 단어(불용어)를 제외할 지 여부 (bool, optional, True|False, default: True)</li>
                <li>"threshold": 추출할 키워드의 유사도 수치의 최소값 (float, optional, default: 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __5. 두 방법 결합하여 사용__

In [13]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT document, label, score
    FROM (
        SEARCH TEXT
        USING nsmc_text_search_model
        OPTIONS (
            search_by='text',
            search_input='가볍게 볼 수 있는 코미디 영화',
            emb_col='convert_result',
            result_col='score',
            top_k=10
            )
        AS
        SELECT *
        FROM nsmc_test
        )
    )

,document,label,score,keyword
0,순수하게 보면 참으로 재밌는 영화,1,0.729469,"{'keyword': ['참으로 재밌는 영화', '순수하게 보면 참으로', '보면 ..."
1,간만에 본 한국영화 중 수작중의 수작,1,0.661832,"{'keyword': ['간만에 한국영화 수작중의', '한국영화 수작중의 수작', ..."
2,오래됐지만 재미있는 영화,1,0.657922,"{'keyword': ['오래됐지만 재미있는 영화', '오래됐지만 재미있는', '오..."
3,코미디로도 사람의 마음을 울릴수 있느 영화,1,0.653881,"{'keyword': ['울릴수 있느 영화', '사람의 마음을 울릴수', '마음을 ..."
4,시간가는줄 모르고 잼있게봤다 볼만한 영화,1,0.642639,"{'keyword': ['시간가는줄 모르고 잼있게봤다', '잼있게봤다 볼만한 영화'..."
5,보고나면 기분 좋은 영화,1,0.639866,"{'keyword': ['보고나면 기분 좋은', '기분 좋은 영화', '보고나면 기..."
6,레전드란 말이 어울리는 몇 안되는 영화중 하나,1,0.635468,"{'keyword': ['레전드란 말이 어울리는', '안되는 영화중 하나', '영화..."
7,다시보고 싶은영화로 추천,1,0.634664,"{'keyword': ['다시보고 싶은영화로 추천', '다시보고 싶은영화로', '싶..."
8,감동적인 영화,1,0.630047,"{'keyword': ['감동적인 영화', '감동적인', '영화'], 'score'..."
9,최고의 영화중 한편,1,0.629051,"{'keyword': ['최고의 영화중 한편', '최고의 영화중', '영화중 한편'..."


In [14]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT document, label, score
        FROM (
            SEARCH TEXT
            USING nsmc_text_search_model
            OPTIONS (
                search_by='text',
                search_input='가볍게 볼 수 있는 코미디 영화',
                emb_col='convert_result',
                result_col='score',
                top_k=10
                )
            AS
            SELECT *
            FROM nsmc_test
            )
        )
    )

,document,label,keywords,score
0,순수하게 보면 참으로 재밌는 영화,1,"[참으로 재밌는 영화, 순수하게 보면 참으로, 보면 참으로 재밌는, 순수하게 보면,...","[0.8065, 0.7946, 0.7799, 0.7604, 0.5688]"
1,간만에 본 한국영화 중 수작중의 수작,1,"[간만에 한국영화 수작중의, 한국영화 수작중의 수작, 한국영화 수작중의, 간만에 한...","[0.9223, 0.8626, 0.8524, 0.7885, 0.6668]"
2,오래됐지만 재미있는 영화,1,"[오래됐지만 재미있는 영화, 오래됐지만 재미있는, 오래됐지만, 재미있는, 영화]","[1.0, 0.8742, 0.7636, 0.616, 0.6072]"
3,코미디로도 사람의 마음을 울릴수 있느 영화,1,"[울릴수 있느 영화, 사람의 마음을 울릴수, 마음을 울릴수 있느, 코미디로도 사람의...","[0.8101, 0.7876, 0.7691, 0.7317, 0.4217]"
4,시간가는줄 모르고 잼있게봤다 볼만한 영화,1,"[시간가는줄 모르고 잼있게봤다, 잼있게봤다 볼만한 영화, 잼있게봤다 볼만한, 모르고...","[0.9144, 0.8888, 0.8589, 0.8081, 0.6883]"
5,보고나면 기분 좋은 영화,1,"[보고나면 기분 좋은, 기분 좋은 영화, 보고나면 기분, 보고나면, 영화]","[0.9262, 0.8887, 0.8331, 0.7309, 0.5361]"
6,레전드란 말이 어울리는 몇 안되는 영화중 하나,1,"[레전드란 말이 어울리는, 안되는 영화중 하나, 영화중, 말이 어울리는 안되는, 하나]","[0.7696, 0.7329, 0.6155, 0.4614, 0.3926]"
7,다시보고 싶은영화로 추천,1,"[다시보고 싶은영화로 추천, 다시보고 싶은영화로, 싶은영화로 추천, 다시보고, 추천]","[1.0, 0.9433, 0.8826, 0.7019, 0.4693]"
8,감동적인 영화,1,"[감동적인 영화, 감동적인, 영화]","[1.0, 0.8893, 0.6404]"
9,최고의 영화중 한편,1,"[최고의 영화중 한편, 최고의 영화중, 영화중 한편, 최고의, 한편]","[1.0, 0.9546, 0.8934, 0.6689, 0.5884]"


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|text|audio|video]</strong>" 쿼리 구문은 검색하고자 하는 이미지|텍스트|오디오|비디오 파일을 정의합니다.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __6. 튜토리얼을 마치며__

이번 튜토리얼에서는 nsmc 영화 리뷰 데이터 세트를 사용하여 텍스트 수치화와 수치화 결과를 바탕으로한 유사 텍스트 검색, 키워드 추출을 진행해 보았습니다. 이번 튜토리얼에서는 모델의 수치화 성능보다는 작동 위주의 설명으로 진행하였습니다. 나만의 텍스트 수치화 모델을 만들어 다양한 형태의 비정형 데이터 세트에 검색 기능을 추가하고 AutoML 기법을 이용한 나만의 모델을 배포할 수 있습니다.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>유사 텍스트 검색 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>